<a href="https://colab.research.google.com/github/janastadtruckerova/wine_quality_classification/blob/main/wine_quality_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predikcia kvality vína**

Cieľom projektu je klasifikovať kvalitu vína ako nadpriemernú alebo podpriemernú na základe fyzikálno-chemických vlastností. Použili sme  dataset Wine Quality, ktorý obsahuje údaje ako napr. obsah alkoholu, kyslosti, cukru, pH a ďalšie premenné.

**Import knižníc a načítanie datasetu**

In [ ]:
# Importovanie potrebných knižníc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
# Načítanie datasetu
data = pd.read_csv("WineQT.csv")
# Zobrazenie prvých 5 riadkov datasetu
print(data.head())

**Preskúmanie datasetu**

In [ ]:
# Štatistický prehľad dát
print(data.describe())

# Kontrola chýbajúcich hodnôt
print(data.isnull().sum())

In [ ]:
data.info()

# **Exploračná analýza**

*Histogramy pre všetky vstupné premenné*

In [ ]:
# Vytvorenie binárnej premennej  kvalita, ak ešte neexistuje
avg_quality = data['quality'].mean()
data['quality_binary'] = (data['quality'] > avg_quality).astype(int)

# Histogramy bez 'quality' a 'quality_binary'
data_features = data.drop(['quality', 'quality_binary'], axis=1)

data_features.hist(bins=15, figsize=(15, 10), color='skyblue', edgecolor='black')
plt.suptitle('Histogramy všetkých vstupných premenných', fontsize=16)
plt.tight_layout()
plt.show()


*Histogram: kvalita vína*

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(x='quality', data=data, palette='viridis')
plt.title('Rozdelenie kvality vína')
plt.xlabel('Kvalita')
plt.ylabel('Počet vzoriek')
plt.grid(axis='y')
plt.show()


*Histogram s rozdelením podľa triedy: pod vs. nad priemerom*

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(data=data, x='alcohol', hue='quality_binary', bins=20, kde=True, palette=['red', 'green'])
plt.title('Rozdelenie alkoholu podľa kvality vína (0 = podpriemerné, 1 = nadpriemerné)')
plt.xlabel('Alkohol')
plt.ylabel('Počet vzoriek')
plt.grid()
plt.show()


*Boxploty – porovnanie vlastností podľa kvality (binárne)*

In [ ]:
import seaborn as sns

# Výber premenných
features = ['alcohol', 'volatile acidity', 'citric acid', 'sulphates', 'pH']

plt.figure(figsize=(15, 10))
for i, col in enumerate(features, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(x='quality_binary', y=col, data=data, palette='Set2')
    plt.title(f'{col} podľa kvality')
    plt.xlabel('Kvalita (0 = podpriemer, 1 = nadpriemer)')
    plt.tight_layout()


 *Pairplot – vzťahy medzi viacerými premennými + farebne podľa kvality*

In [ ]:
# Pre jednoduchosť vyberieme len niekoľko premenných
selected = data[['alcohol', 'volatile acidity', 'citric acid', 'sulphates', 'quality_binary']]

sns.pairplot(selected, hue='quality_binary', palette='Set1', diag_kind='kde')
plt.suptitle('Pairplot vybraných vlastností podľa kvality', y=1.02)
plt.show()


*Korelácie – ako silne súvisia premenné s kvalitou*

In [ ]:
# Korelácia s binárnou kvalitou
correlations = data.corr()['quality_binary'].sort_values(ascending=False)

print('Top korelujúce vlastnosti s nadpriemernou kvalitou:')
print(correlations)

# Vizualizácia
plt.figure(figsize=(10, 6))
sns.barplot(x=correlations.values, y=correlations.index, palette='coolwarm')
plt.title('Korelácia premenných s binárnou kvalitou')
plt.xlabel('Korelačný koeficient')
plt.grid()
plt.show()


# **Predspracovanie dát**

Predspracujeme dáta: rozdelenie na tréningové a testovacie množiny, a štandardizácia.




In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# Výpočet priemernej kvality
avg_quality = data['quality'].mean()

print(f'Priemerná kvalita: {avg_quality:.2f}')

# Vytvorenie binárnej cieľovej premennej
data['quality_binary'] = (data['quality'] > avg_quality).astype(int)

# Rozdelenie dát na vstupy a výstupy
X = data.drop(['quality', 'quality_binary'], axis=1)
y = data['quality_binary']

# Rozdelenie dát na tréningové a testovacie množiny
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Štandardizácia vstupov
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)




In [ ]:
# Trénovanie logistickej regresie
model = LogisticRegression()
model.fit(X_train, y_train)

# Predikcia
y_pred = model.predict(X_test)

# Vyhodnotenie modelu
print(f'Presnosť: {accuracy_score(y_test, y_pred):.4f}')
print('Maticu zámien:')
print(confusion_matrix(y_test, y_pred))
print('Klasifikačná správa:')
print(classification_report(y_test, y_pred))

# **Vizualizácie - Vyhodnotenie modelu**

*Confusion Matrix*

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix – Logistická regresia')
plt.show()


 *ROC krivka a AUC skóre*

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

y_probs = model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_probs)
auc_score = roc_auc_score(y_test, y_probs)

plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, label=f'Logistická regresia (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], 'k--', label='Náhodná klasifikácia')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC krivka')
plt.legend()
plt.grid()
plt.show()


*Koeficienty/váhy modelu – význam vstupných premenných*

In [ ]:
import pandas as pd
coefs = pd.Series(model.coef_[0], index=X.columns)
coefs.sort_values().plot(kind='barh', figsize=(8, 6), color='teal')
plt.title('Váhy vstupných premenných (logistická regresia)')
plt.xlabel('Hodnota koeficientu')
plt.grid()
plt.show()
